# 라이브러리 불러오기

In [1]:
import pandas as pd
import requests
import xmltodict
import time
import warnings
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.interval import IntervalTrigger
from datetime import datetime
from pytz import timezone

# 건대입구역 부근 교통, 인구 정보 가져오기

In [15]:
road_traffic = pd.DataFrame()

# 경고메시지 무시
warnings.filterwarnings(action='ignore')

# API 호출
region = input("어느 지역을 검색하시겠습니까? : ")
print("정보 수집을 시작합니다.")

def extract_link():
    global road_traffic
    url = 'http://openapi.seoul.go.kr:8088/{api_key}' + region
    try:
        response = requests.get(url)
    except requests.exceptions.RequestException as e:
        print("예외 발생 : ", e)
        print("인터넷 연결이 끊어졌습니다. 10초 후에 다시 시도합니다.")
        time.sleep(300)
        extract_link()
        return
    
    data = xmltodict.parse(response.content)
    
    # 데이터프레임 생성
    df = pd.DataFrame(data["SeoulRtd.citydata"]["CITYDATA"])

    # 결과 확인
    ppltn = pd.DataFrame(df['LIVE_PPLTN_STTS']['LIVE_PPLTN_STTS'], index = [0]) ## 인구 정보
    df = df["ROAD_TRAFFIC_STTS"]["ROAD_TRAFFIC_STTS"][:]
    df = pd.DataFrame(df)
    ppltn = pd.concat([ppltn]*df.shape[0], ignore_index=True)
    df = pd.concat([df, ppltn], axis = 1) # 인구 정보 추가
    df = df.set_index("PPLTN_TIME")
    road_traffic = road_traffic.append(df)
    print("-"*30, "API 호출됨", "-"*30, datetime.now(timezone("Asia/Seoul")))
    
def get_current_time_str() : 
    return datetime.now(timezone("Asia/Seoul")).strftime("%Y-%m-%d_%H-%M")

def per8hours():
    global road_traffic
    #road_traffic.set_index('PPLTN_TIME')
    road_traffic.to_excel(f"Konkuk_traffic_{get_current_time_str()}.xlsx")
    road_traffic = pd.DataFrame() # road_traffic 초기화
    print("Results saved successfully.")

sched = BackgroundScheduler()

extract_link()

# 5분 단위로 api 호출
job1 = sched.add_job(extract_link, trigger=IntervalTrigger(minutes= 5), id="job1")

# 4시간 단위로 데이터프레임 초기화 및 저장
job2 = sched.add_job(per8hours, 'interval', hours = 4, id='job2')

# 스케줄러 시작
sched.start()

while True:
    command = input("정보 수집을 중단하시겠습니까? (Y/N): ")
    if command.lower() == "y":
        sched.remove_job("job1")
        sched.shutdown()
        break
    else:
        continue

어느 지역을 검색하시겠습니까? : 건대입구역
정보 수집을 시작합니다.
------------------------------ API 호출됨 ------------------------------ 2023-04-19 23:46:37.504486+09:00
------------------------------ API 호출됨 ------------------------------ 2023-04-19 23:51:39.399145+09:00
------------------------------ API 호출됨 ------------------------------ 2023-04-19 23:56:38.977041+09:00
------------------------------ API 호출됨 ------------------------------ 2023-04-20 00:01:38.949714+09:00
------------------------------ API 호출됨 ------------------------------ 2023-04-20 00:06:38.928331+09:00
------------------------------ API 호출됨 ------------------------------ 2023-04-20 00:11:38.716891+09:00
------------------------------ API 호출됨 ------------------------------ 2023-04-20 00:16:38.649439+09:00
------------------------------ API 호출됨 ------------------------------ 2023-04-20 00:21:38.585450+09:00
------------------------------ API 호출됨 ------------------------------ 2023-04-20 00:26:38.572216+09:00
------------------------------ API

Run time of job "extract_link (trigger: interval[0:05:00], next run at: 2023-04-20 14:46:37 KST)" was missed by 0:00:01.985384


------------------------------ API 호출됨 ------------------------------ 2023-04-20 14:46:38.685105+09:00
------------------------------ API 호출됨 ------------------------------ 2023-04-20 14:51:38.520934+09:00
------------------------------ API 호출됨 ------------------------------ 2023-04-20 14:56:38.584681+09:00
------------------------------ API 호출됨 ------------------------------ 2023-04-20 15:01:39.111976+09:00
------------------------------ API 호출됨 ------------------------------ 2023-04-20 15:06:38.580329+09:00
------------------------------ API 호출됨 ------------------------------ 2023-04-20 15:11:38.991675+09:00
------------------------------ API 호출됨 ------------------------------ 2023-04-20 15:16:38.937943+09:00
------------------------------ API 호출됨 ------------------------------ 2023-04-20 15:21:38.881134+09:00
------------------------------ API 호출됨 ------------------------------ 2023-04-20 15:26:39.068779+09:00
------------------------------ API 호출됨 ------------------------------ 202

In [16]:
road_traffic

,LINK_ID,ROAD_NM,START_ND_CD,START_ND_NM,START_ND_XY,END_ND_CD,END_ND_NM,END_ND_XY,DIST,SPD,...,PPLTN_RATE_10,PPLTN_RATE_20,PPLTN_RATE_30,PPLTN_RATE_40,PPLTN_RATE_50,PPLTN_RATE_60,PPLTN_RATE_70,RESNT_PPLTN_RATE,NON_RESNT_PPLTN_RATE,REPLACE_YN
PPLTN_TIME,,,,,,,,,,,,,,,,,,,,,
2023-04-20 23:10,1030006700,아차산로,1040017700,성수사거리,127.0639970537829839_37.5421849901295275,1030002200,성수역앞교차로,127.0571187104605571_37.5442656148544600,650,25.0,...,9.3,43.9,16.8,10.2,9.1,5.6,5.1,28.9,71.1,N
2023-04-20 23:10,1030006800,아차산로,1030002200,성수역앞교차로,127.0571187104605571_37.5442656148544600,1040017700,성수사거리,127.0639970537829839_37.5421849901295275,650,37.0,...,9.3,43.9,16.8,10.2,9.1,5.6,5.1,28.9,71.1,N
2023-04-20 23:10,1030025300,연무장길,1030010700,광림물산,127.0562670206634550_37.5421881695674244,1040016700,삼보약국,127.0628517337154761_37.5401974323451881,622,15.0,...,9.3,43.9,16.8,10.2,9.1,5.6,5.1,28.9,71.1,N
2023-04-20 23:10,1040001303,동일로,1040016700,삼보약국,127.0628517337154761_37.5401974323451881,1040017700,성수사거리,127.0639970537829839_37.5421849901295275,243,36.0,...,9.3,43.9,16.8,10.2,9.1,5.6,5.1,28.9,71.1,N
2023-04-20 23:10,1040001402,동일로,1040016700,삼보약국,127.0628517337154761_37.5401974323451881,1040015800,영동대교고가차도북측,127.0624393471961326_37.5394740689106641,89,32.5,...,9.3,43.9,16.8,10.2,9.1,5.6,5.1,28.9,71.1,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-21 00:20,1040029100,아차산로.인접길,1040015300,폴바셋더클래식500점,127.0718863749377903_37.5386776459314149,1040016100,이디야커피건대스타시티2호점,127.0723778850670982_37.5393435180570023,85,11.0,...,9.2,45.0,15.3,8.9,9.1,6.5,5.7,38.1,61.9,N
2023-04-21 00:20,1040029200,동일로18길,1040014700,광진문화예술회관,127.0696747554783457_37.5378877582848389,1040016700,삼보약국,127.0628517337154761_37.5401974323451881,655,12.0,...,9.2,45.0,15.3,8.9,9.1,6.5,5.7,38.1,61.9,N
2023-04-21 00:20,1040029300,동일로18길,1040016700,삼보약국,127.0628517337154761_37.5401974323451881,1040014700,광진문화예술회관,127.0696747554783457_37.5378877582848389,655,18.0,...,9.2,45.0,15.3,8.9,9.1,6.5,5.7,38.1,61.9,N


In [17]:
road_traffic.to_excel("Konkuk_traffic_2023-04-20_01-00.xlsx")